## Portfolio Analyses

We would need both the total ha for each landscape, plus the total ha of all cohort 2 (landscapes cohort) projects. I realize this will likely be a fairly small number though, by only counting the hectares of the project as monitored.  
GCB: Landscape boundary = 11,655,440.106 ha, sub-landscape boundary = 2,976,149.2 ha  
LKR: Landscape boundary = 2,579,929 ha, sub-landscape boundary = 983,000 ha  
GRV: Landscape boundary = 15,874,367 ha, sub-landscape boundary = 6,502,087 ha

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import re
import os
import requests
import yaml
import json
import sys
sys.path.append('../src/')

# data gathering
import process_api_results as clean
import api_utils as tm
from tm_api_utils import pull_tm_api_data
%load_ext autoreload
%autoreload 2

In [4]:
df = pd.read_csv("../data/feats/tm_api_c1_07-14-2025.csv")
mangrove = df[df.target_sys == 'mangrove']
mangrove.shape

(48, 17)

In [7]:
mangrove.project_name.value_counts()

project_name
TAZ_22_FT          13
GHA_22_FOTE        10
BEN_22_ECOBEN       7
CIV_22_IMPACAD      7
MOZ_22_ISTITUTO     7
GHA_22_HMPOANO      4
Name: count, dtype: int64

# Total ha of each landscape

# Total ha of cohort 2 projects

In [8]:
tm_prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" 
response_outfile = f"../data/tm_api_response/restore_local_report.json"
c2_feats = f'../data/restore_local_report.csv'   
tm_auth_path = '../secrets.yaml'

In [9]:
with open(tm_auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [12]:
# calculate total ha of cohort 2 projects
approved_projects = '../terrafund_approved_projects_202504021529.csv'
df = pd.read_csv(approved_projects)
cohort2 = df[(df.cohort == 'terrafund-landscapes')]
c2_ids = list(set(cohort2.project_id))
len(c2_ids)

76

In [22]:
df.head()

,project_id,project_name,country,cohort,landscape,framework_key
0,c462918b-47f7-4ed5-99e0-7fec6e342036,"""Nakuru Eco-Reforestation Project""",KE,terrafund-landscapes,Greater Rift Valley of Kenya,enterprises
1,617601e0-9839-49fd-b48e-6c07404e7140,Afram Headwaters Restoration Initiative (AHRI),GH,terrafund-landscapes,Ghana Cocoa Belt,enterprises
2,943bb150-f1b7-4ad2-bb9e-60a559df2ebd,Agriculture and forest landscape restoration -...,GH,terrafund,Ghana Cocoa Belt,terrafund
3,0f4bdbba-adf9-4554-ba29-fb28d3b9c1fd,AGRICULTURE AND FOREST LANDSCAPES RESTORATION ...,GH,terrafund-landscapes,Ghana Cocoa Belt,terrafund-landscapes
4,47f07709-b503-4eeb-98fa-aea62e979ca8,Agroecology Project - Centre ValBio,MG,terrafund,NaN,terrafund


In [13]:
c2_results = tm.pull_wrapper(tm_prod_url, headers, c2_ids, outfile=response_outfile)

Pulling Projects: 100%|████████████████████████████████████████████████████████████████████████████| 76/76 [06:02<00:00,  4.77s/project]


Results saved to ../data/tm_api_response/restore_local_report.json


In [17]:
c2_response_clean = clean.process_tm_api_results(c2_results,
                                                 outfile1=c2_feats, 
                                                 outfile2=None)

/Users/jessica.ertel/github/terrafund-portfolio-analyses/notebooks/../src/process_api_results.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  affected_rows.loc[is_feb_29] = non_leap_years
/Users/jessica.ertel/github/terrafund-portfolio-analyses/notebooks/../src/process_api_results.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  affected_rows.loc[is_feb_29] = non_leap_years


Number of rows missing a 'plantstart' date: 0/17235
Number of rows missing a 'plantend' date: 1/17235
⚠️ Total rows missing start and end plant date: 0
⚠️ Total projects missing 'plantstart': 0
⚠️ Total polygons missing 'plantstart': 0
Projects fully removed: 0
Projects partially affected: 0


In [18]:
c2_response_clean.head()

,project_id,poly_id,site_id,geometry,plantstart,plantend,practice,target_sys,dist,project_phase,area,ttc_2023,ttc_2024,ttc_2022
0,24e290f1-5b13-4ca8-acce-43dcc8e5bb43,5b61a20a-f927-4aab-952a-51cce0e52514,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.19404...",2024-06-29,2024-06-29,tree-planting,agroforest,single-line,,2.881584,81.0,NaN,NaN
1,24e290f1-5b13-4ca8-acce-43dcc8e5bb43,150358b3-b0d1-480d-994f-e33b0686fc83,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.19920...",2024-06-16,2024-06-16,tree-planting,agroforest,single-line,,0.517322,92.0,NaN,NaN
2,24e290f1-5b13-4ca8-acce-43dcc8e5bb43,8a72c943-e357-4214-bfa4-d13681ec3e68,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.18773...",2024-06-15,2024-06-15,tree-planting,agroforest,single-line,,2.881558,76.0,NaN,NaN
3,24e290f1-5b13-4ca8-acce-43dcc8e5bb43,fb0d0abc-ec12-40b8-9450-cc317b10d48a,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.20570...",2024-06-15,2024-06-15,tree-planting,agroforest,single-line,,5.175160,89.0,NaN,NaN
4,24e290f1-5b13-4ca8-acce-43dcc8e5bb43,abb8527e-2b5d-4b5a-bfc9-20cf2ef1ecd8,7e9691c1-e4be-498c-8a3a-6a2b98398a0f,"{'type': 'Polygon', 'coordinates': [[[-1.20172...",2024-06-22,2024-06-22,tree-planting,agroforest,single-line,,0.961935,82.0,NaN,NaN


In [19]:
total = c2_response_clean.area.sum()
total

np.float64(13941.512284103412)